Install if not available yet

In [3]:
pip install pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


Import libraries

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import transformers
from transformers import TFBertModel, BertTokenizer
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

d:\Capstone\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\syari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import tensorflow as tf
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num CPUs Available:  1
Num GPUs Available:  1


In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
# Load your dataset
data = pd.read_csv('../Data/podcasts_data.csv')
data = data.iloc[:, :2]
data.columns = ['Genre', 'Podcast Name']
data.head()


,Genre,Podcast Name
0,arts and entertainment,Easy Stories in English
1,arts and entertainment,Podcast Buku Kutu
2,arts and entertainment,Underwood and Flinch and Other Audiobooks by M...
3,arts and entertainment,Podcast Resensi Buku
4,arts and entertainment,SupremeMasterTV


In [5]:
# Separate the columns into different DataFrames
genres = data[['Genre']].drop_duplicates().reset_index(drop=True)
podcasts = data[['Podcast Name']].drop_duplicates().reset_index(drop=True)

# Save each DataFrame to a CSV file
genres.to_csv('genres.csv', index=False)
podcasts.to_csv('podcasts.csv', index=False)

print("Separated files 'genres.csv' and 'podcasts.csv' have been created.")

Separated files 'genres.csv' and 'podcasts.csv' have been created.


In [6]:
genres = pd.read_csv('./genres.csv')
podcasts = pd.read_csv('./podcasts.csv')

print("Variable have been called")

Variable have been called


In [7]:
# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
#BERT Embeddings
def get_bert_embeddings_batch(text_list, tokenizer, model, batch_size):
    batch_embeddings = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors='tf', padding=True, truncation=True)
        outputs = model(inputs)
        batch_embeddings.append(tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy())
    return np.concatenate(batch_embeddings)

genres_embeddings = get_bert_embeddings_batch(genres['Genre'].tolist(), tokenizer, model, batch_size=128)
podcasts_embeddings = get_bert_embeddings_batch(podcasts['Podcast Name'].tolist(), tokenizer, model, batch_size=128)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

def search(query, tokenizer, model, data_embeddings, data_labels, top_k=5):
    query_embedding = get_bert_embeddings_batch([query], tokenizer, model, batch_size=1)
    similarities = cosine_similarity(query_embedding, data_embeddings).flatten()
    top_k_indices = similarities.argsort()[-top_k:][::-1]
    return [(data_labels[idx], similarities[idx]) for idx in top_k_indices]



In [10]:
# Example search
query = "easy english"
results = search(query, tokenizer, model, podcasts_embeddings, podcasts['Podcast Name'].tolist())
for name, score in results:
    print(f"Podcast Name: {name} , Similarity: {score:.4f}")

Podcast Name: English 102: Genre Remediation , Similarity: 0.6366
Podcast Name: Easy Stories in English , Similarity: 0.6311
Podcast Name: Limited Lexicon , Similarity: 0.6298
Podcast Name: Speculative Fiction Writing Made Simple , Similarity: 0.6258
Podcast Name: Lit In 9ja , Similarity: 0.6122


In [11]:
def search_podcasts_by_genre(genre_query, tokenizer, model, genres_embeddings, podcasts_embeddings, genres_labels, podcasts_labels, top_k=5):
    genre_results = search(genre_query, tokenizer, model, genres_embeddings, genres_labels, top_k=1)
    top_genre, _ = genre_results[0]
    print(f"Top Genre: {top_genre}")

    results = search(top_genre, tokenizer, model, podcasts_embeddings, podcasts_labels, top_k)
    for name, score in results:
        print(f"Podcast Name: {name}, Similarity: {score:.4f}")

# Example search
search_podcasts_by_genre("Religion", tokenizer, model, genres_embeddings, podcasts_embeddings, genres['Genre'].tolist(), podcasts['Podcast Name'].tolist())

Top Genre: hobbies
Podcast Name: C.O.O.L. Radio, Similarity: 0.5270
Podcast Name: Audiobooks with Subrata, Similarity: 0.5028
Podcast Name: Hobbies, Similarity: 0.4926
Podcast Name: Reel Friends: A Movie Podcast, Similarity: 0.4918
Podcast Name: FO Paris Podcasts, Similarity: 0.4908
